In [2]:
import sys, os
sys.path.append('..')
from Data.TimeSeries import *

from Data import factors
import Quandl
import pandas as pd
import matplotlib

%matplotlib inline

## 0. downloading data
We download all the ETF data and save them intoa  HDF5 file

In [5]:
tickers = ['GOOG/NYSE_SPY', #S&P 500 ETF 
           'GOOG/AMEX_EWJ', # iShares MSCI Japan ETF
           'GOOG/NYSE_IEV', # iShares Europe ETF
           'GOOG/NYSE_VWO', # Vanguard Emerging Market Stock ETF
           
           #'GOOG/NYSE_VNQ', # Vanguard MSCI US Reits
           'GOOG/NYSE_IYR', # iShares U.S. Real Estate ETF
           'GOOG/NYSE_RWX', # SPDR DJ Wilshire Intl Real Estate ETF
           
           'GOOG/NYSEARCA_TLT',  # 20 Years Treasury ETF
           'GOOG/NYSEARCA_TLH',  # 15-20 Years Treasury
           
           'GOOG/AMEX_GSG', # GSCI Commodity-Indexed Trust Fund
           'GOOG/NYSEARCA_GLD',  # SPDR Gold ETF
           
          ]

settings = Settings()
df_price, df_ret = TimeSeries(settings).get_agg_data(tickers)
    

# Testng Trend Following Approach on ETFs
##  1. Meb 10 Month MA Strategy
meb faber has written a paper illustrating a trend-following strategy: 10 months MA of SPY as the momentum. If the price cross-up or cross-down the 10MMA, buy or sell accordingly. 

In [ ]:
df_spy_m = Quandl.get('GOOG/NYSE_SPY', collapse='monthly')
df_spy_d = Quandl.get('GOOG/NYSE_SPY', collapse='daily')                      

In [ ]:
data = pd.DataFrame()

data['price'] = df_spy_m['Close']
data['10MMA'] = pd.rolling_mean(data['price'], 10)
data['ret'] = data['price'].pct_change()
data['pos'] = (data['price'] > data['10MMA']).shift(1) * 1 
data['ret_ma'] = data['ret'] * (data['price'] > data['10MMA']).shift(1)
data = data.dropna()
data.index = data.index.tz_localize('UTC')

In [ ]:
(1+data[['ret', 'ret_ma']]).cumprod().plot(figsize=[20, 8])

As the previous test is on monthly price and return, we cannot see the details within the month. Therefore, we now use daily price/return, together with the position information derived from the 10MMA strategy. 

In [ ]:
import pyfolio as py
data_d = pd.DataFrame()
data_d['price'] = df_spy_d['Close']
data_d['ret'] = data_d['price'].pct_change()
data_d.index = data_d.index.tz_localize('UTC')

data_d = data_d.join(data['pos'], how='outer')
data_d = data_d.fillna(method='bfill')
data_d['ret_ma'] = data_d['ret'] * data_d['pos']
data_d = data_d.dropna()

In [ ]:
(1+data_d[['ret','ret_ma']]).cumprod().plot(figsize=[20, 8])

In [ ]:
py.show_perf_stats(data_d['ret_ma'], data_d['ret'])
#(data['ret_ma'])

## 1.1 ETF Rotation with 10 MMA
To extend the 10MMA strategy, we use 2 ETFs: SPY and TLT (20year treasury). When SPY cross-down 10MMA, instead of keeping in cash, we rotate into TLT. 

In [ ]:
df_tlt = Quandl.get('GOOG/NYSEARCA_TLT')
df_tlt.index = df_tlt.index.tz_localize('UTC')

In [ ]:
data_11 = pd.DataFrame()
data_11['spy'] = data_d['price']
data_11.index = data_11.index.tz_convert('UTC')

data_11 = data_11.join(df_tlt['Close'], how='inner')
data_11 = data_11.rename(columns={'Close':'tlt'})

In [ ]:
data_11['ret_spy'] = data_11['spy'].pct_change()
data_11['ret_tlt'] = data_11['tlt'].pct_change()
data_11['pos'] = data_d['pos']

# strategy 1: rotating between SPY and TLT
data_11['ret_rotate'] = data_11['ret_spy'] * data_11['pos'] + data_11['ret_tlt'] * (1-data_11['pos'])
# Strategy 0: MMA strtegy
data_11['ret_spy_ma'] = data_11['ret_spy'] * data_11['pos']
# Strategy 2: Equal Weights Portfolio
data_11['ret_1/2'] = (data_11['ret_spy']*0.7 + data_11['ret_tlt'] * 0.3) #/ 2

we can see in the following figure, the rotation strategy out-perform the rest strategies, which also shows the negative correlation between TLT and SPX


In [ ]:
(1+data_11[['ret_spy', 'ret_tlt', 'ret_rotate', 'ret_spy_ma', 'ret_1/2']]).cumprod().plot(figsize=[20, 10])

## 1.2 SPY, GLD, TLT


In [ ]:
df_gld = Quandl.get('GOOG/NYSEARCA_GLD')
df_gld.index = df_gld.index.tz_localize('UTC')

In [ ]:
data_12 = data_11.copy()
data_12['gld'] = df_gld['Close']
data_12['ret_gld'] = data_12['gld'].pct_change()

In [ ]:
data_12['ret_1/3'] = (data_12['ret_gld'] + data_12['ret_spy'] + data_12['ret_tlt'] ) /3
data_12 = data_12.dropna()
(1+data_12[['ret_1/3', 'ret_1/2', 'ret_spy', 'ret_rotate']]).cumprod().plot(figsize=[20, 10])

In [ ]:
data_11